# Task 1

# Data Cleaning
Now since we have extracted data from the website, it is not cleaned and ready to be analyzed yet. The reviews section will need to be cleaned for punctuations, spellings and other characters.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

In [2]:
#create a dataframe from csv file

cwd = os.getcwd()

df = pd.read_csv(cwd+"/BA_reviews.csv", index_col=0)


In [3]:
df.head()

,reviews,stars,date,country
0,Not Verified | They changed our Flights from ...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,18th April 2023,United States
1,Not Verified | At Copenhagen the most chaotic...,2,18th April 2023,United States
2,✅ Trip Verified | Worst experience of my life...,5,17th April 2023,United States
3,✅ Trip Verified | Due to code sharing with Ca...,1,17th April 2023,Hong Kong
4,✅ Trip Verified | LHR check in was quick at t...,3,16th April 2023,United Kingdom


In [4]:
df.info

<bound method DataFrame.info of                                                 reviews  \
0     Not Verified |  They changed our Flights from ...   
1     Not Verified |  At Copenhagen the most chaotic...   
2     ✅ Trip Verified |  Worst experience of my life...   
3     ✅ Trip Verified |  Due to code sharing with Ca...   
4     ✅ Trip Verified |  LHR check in was quick at t...   
...                                                 ...   
3495  Once again a good flight from LHR to Warsaw in...   
3496  LGW-MRU-LGW in business. Outbound flight good ...   
3497  LHR-FRA-LCY May 2014. LHR-FRA on a 767 row 18....   
3498  Rating : 10/10 Cabin Flown Economy Value for M...   
3499  08/5/2014 JFK-LHR-BRU. After the perfect outbo...   

                              stars             date         country  
0     \n\t\t\t\t\t\t\t\t\t\t\t\t\t5  18th April 2023   United States  
1                                 2  18th April 2023   United States  
2                                 5  17th Apri

In [5]:
df.describe

<bound method NDFrame.describe of                                                 reviews  \
0     Not Verified |  They changed our Flights from ...   
1     Not Verified |  At Copenhagen the most chaotic...   
2     ✅ Trip Verified |  Worst experience of my life...   
3     ✅ Trip Verified |  Due to code sharing with Ca...   
4     ✅ Trip Verified |  LHR check in was quick at t...   
...                                                 ...   
3495  Once again a good flight from LHR to Warsaw in...   
3496  LGW-MRU-LGW in business. Outbound flight good ...   
3497  LHR-FRA-LCY May 2014. LHR-FRA on a 767 row 18....   
3498  Rating : 10/10 Cabin Flown Economy Value for M...   
3499  08/5/2014 JFK-LHR-BRU. After the perfect outbo...   

                              stars             date         country  
0     \n\t\t\t\t\t\t\t\t\t\t\t\t\t5  18th April 2023   United States  
1                                 2  18th April 2023   United States  
2                                 5  17th Ap

In [6]:
df['verified'] = df.reviews.str.contains("Trip Verified")

In [7]:
df['verified']

0       False
1       False
2        True
3        True
4        True
        ...  
3495    False
3496    False
3497    False
3498    False
3499    False
Name: verified, Length: 3500, dtype: bool

# Cleaning Reviews
We will extract the column of reviews into a separate dataframe and clean it for semantic analysis

In [8]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()


In [9]:
reviews_data = df.reviews.str.strip("✅ Trip Verified |")

#create an empty list to collect cleaned data corpus
corpus =[]

In [10]:
#loop through each review, remove punctuations, small case it, join it and add it to corpus
for rev in reviews_data:
    rev = re.sub('[^a-zA-Z]',' ', rev)
    rev = rev.lower()
    rev = rev.split()
    rev = [lemma.lemmatize(word) for word in rev if word not in set(stopwords.words("english"))]
    rev = " ".join(rev)
    corpus.append(rev)

In [11]:
# add the corpus to the original dataframe

df['corpus'] = corpus

In [12]:
df.head()

,reviews,stars,date,country,verified,corpus
0,Not Verified | They changed our Flights from ...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,18th April 2023,United States,False,verified changed flight brussels london heathr...
1,Not Verified | At Copenhagen the most chaotic...,2,18th April 2023,United States,False,verified copenhagen chaotic ticket counter ass...
2,✅ Trip Verified | Worst experience of my life...,5,17th April 2023,United States,True,worst experience life trying deal customer ser...
3,✅ Trip Verified | Due to code sharing with Ca...,1,17th April 2023,Hong Kong,True,due code sharing cathay pacific downgraded ba ...
4,✅ Trip Verified | LHR check in was quick at t...,3,16th April 2023,United Kingdom,True,lhr check quick first wing quickly security fi...


In [13]:
#Cleaning/Fromat date
df.dtypes

reviews     object
stars       object
date        object
country     object
verified      bool
corpus      object
dtype: object

In [14]:
# convert the date to datetime format

df.date = pd.to_datetime(df.date)

In [15]:
df.date.head()

0   2023-04-18
1   2023-04-18
2   2023-04-17
3   2023-04-17
4   2023-04-16
Name: date, dtype: datetime64[ns]

In [16]:
#Cleaning ratings with stars
#check for unique values
df.stars.unique()

array(['\n\t\t\t\t\t\t\t\t\t\t\t\t\t5', '2', '5', '1', '3', '4', '9', '7',
       '10', '8', '6', 'None'], dtype=object)

In [17]:
# remove the \t and \n from the ratings
df.stars = df.stars.str.strip("\n\t\t\t\t\t\t\t\t\t\t\t\t\t")

In [17]:
df.stars.value_counts()

1       770
2       393
3       390
8       350
10      308
7       303
9       299
5       261
4       239
6       183
None      4
Name: stars, dtype: int64

In [18]:
# drop the rows where the value of ratings is None
df.drop(df[df.stars == "None"].index, axis=0, inplace=True)
#check the unique values again
df.stars.unique()

array(['5', '2', '1', '3', '4', '9', '7', '10', '8', '6'], dtype=object)

In [19]:
#Check for null Values
df.isnull().value_counts()

reviews  stars  date   country  verified  corpus
False    False  False  False    False     False     3494
                       True     False     False        2
dtype: int64

In [20]:
df.country.isnull().value_counts()

False    3494
True        2
Name: country, dtype: int64

In [21]:
#drop the rows using index where the country value is null
df.drop(df[df.country.isnull() == True].index, axis=0, inplace=True)

In [22]:
df.shape

(3494, 6)

In [24]:
#resetting the index
df.reset_index(drop=True)

,reviews,stars,date,country,verified,corpus
0,Not Verified | They changed our Flights from ...,5,2023-04-18,United States,False,verified changed flight brussels london heathr...
1,Not Verified | At Copenhagen the most chaotic...,2,2023-04-18,United States,False,verified copenhagen chaotic ticket counter ass...
2,✅ Trip Verified | Worst experience of my life...,5,2023-04-17,United States,True,worst experience life trying deal customer ser...
3,✅ Trip Verified | Due to code sharing with Ca...,1,2023-04-17,Hong Kong,True,due code sharing cathay pacific downgraded ba ...
4,✅ Trip Verified | LHR check in was quick at t...,3,2023-04-16,United Kingdom,True,lhr check quick first wing quickly security fi...
...,...,...,...,...,...,...
3489,Once again a good flight from LHR to Warsaw in...,6,2014-05-11,United Kingdom,False,good flight lhr warsaw cabin crew efficient fr...
3490,LGW-MRU-LGW in business. Outbound flight good ...,2,2014-05-11,United Kingdom,False,lgw mru lgw business outbound flight good atte...
3491,LHR-FRA-LCY May 2014. LHR-FRA on a 767 row 18....,9,2014-05-11,United Kingdom,False,lhr fra lcy may lhr fra row checked via mobile...
3492,Rating : 10/10 Cabin Flown Economy Value for M...,8,2014-05-11,United Kingdom,False,rating cabin flown economy value money seat co...


In [ ]:
# Now our data is all cleaned and ready for data visualization and data analysis.

# export the cleaned data

df.to_csv(cwd + "/cleaned-BA-reviews.csv")